In [2]:
import os
import numpy as np

import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


2025-11-03 18:15:16.864324: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 18:15:17.942250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-03 18:15:26.736001: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model = load_model("drowsiness_model5.h5")


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (64, 64))
        face = face / 255.0
        face = np.expand_dims(face, axis=0)

        pred = model.predict(face)[0][0]

        if pred >= 0.5:
            label = "ALERT"
            color = (0, 255, 0)
        else:
            label = "DROWSY"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Driver Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 754ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Mobile Net

In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("mobilenetv2_drowsy_model.keras")

# Load face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        # Resize to match model input
        face = cv2.resize(face, (128, 128))

        # Normalize and expand dimensions
        face = face / 255.0
        face = np.expand_dims(face, axis=0)

        # Predict
        pred = model.predict(face, verbose=0)[0][0]
        print("Prediction:", pred)  # Debug output

        # Since {'DROWSY': 0, 'NATURAL': 1}, model likely predicts prob of being NATURAL
        if pred >= 0.5:
            label = "ALERT"   # class 1
            color = (0, 255, 0)
        else:
            label = "DROWSY"  # class 0
            color = (0, 0, 255)

        # Draw face box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Show the frame
    cv2.imshow("Driver Drowsiness Detection - MobileNetV2", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Prediction: 0.9996804
Prediction: 0.9991976
Prediction: 0.9991914
Prediction: 0.9987469
Prediction: 0.99946994
Prediction: 0.9998929
Prediction: 0.9998585
Prediction: 0.9999736
Prediction: 0.99993813
Prediction: 0.99996036
Prediction: 0.99997044
Prediction: 0.99996424
Prediction: 0.9999557
Prediction: 0.9999838
Prediction: 0.9998503
Prediction: 0.99974126
Prediction: 0.9999505
Prediction: 0.99980783
Prediction: 0.9996557
Prediction: 0.9995399
Prediction: 0.9998711
Prediction: 0.9999025
Prediction: 0.9995426
Prediction: 0.99933267
Prediction: 0.999966
Prediction: 0.9999733
Prediction: 0.9999914
Prediction: 0.9999513
Prediction: 0.9998084
Prediction: 0.9999574
Prediction: 0.9999893
Prediction: 0.99994504
Prediction: 0.99996984
Prediction: 0.99998903
Prediction: 0.99994826
Prediction: 0.99537677
Prediction: 0.9892358
Prediction: 0.99931806
Prediction: 0.99980813
Prediction: 0.9999763
Prediction: 0.9999739
Prediction: 0.999944
Prediction: 0.99996334
Prediction: 0.99956274
Prediction: 0.999

Efficient Net

In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# ---------------------------------------------------------------------
# Load your existing grayscale EfficientNet model
# ---------------------------------------------------------------------
model = load_model("efficientnet_drowsy_model_fixed.keras", safe_mode=False)


# ---------------------------------------------------------------------
# Load Haar cascade for face detection
# ---------------------------------------------------------------------
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------
IMG_SIZE = 128  # EfficientNet input size

# ---------------------------------------------------------------------
# Real-time detection loop
# ---------------------------------------------------------------------
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        # ✅ Convert to GRAYSCALE because model was trained on (128,128,1)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # ✅ Resize to model input size
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))

        # ✅ Normalize
        face = face.astype("float32") / 255.0

        # ✅ Expand dims for (1,128,128,1)
        face = np.expand_dims(face, axis=-1)
        face = np.expand_dims(face, axis=0)

        # ✅ Predict
        pred = model.predict(face, verbose=0)[0][0]

        # ✅ Interpret prediction
        if pred >= 0.5:
            label = "ALERT"    # class 1 (Natural)
            color = (0, 255, 0)
        else:
            label = "DROWSY"   # class 0
            color = (0, 0, 255)

        # ✅ Draw results
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{label} ({pred:.2f})", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Show the frame
    cv2.imshow("Driver Drowsiness Detection - EfficientNet", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: Input 0 of layer "stem_conv" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 129, 129, 1)